In [1]:
import os

os.environ['R_HOME'] = '/home/comp/cszrwang/anaconda3/envs/SPACEL/lib/R'

In [2]:
from SPACEL.setting import set_environ_seed
set_environ_seed()
from SPACEL import Spoint
import scanpy as sc
import numpy as np
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams['font.serif'] = ['Arial']
sc.settings.set_figure_params(dpi=50,dpi_save=300,facecolor='white',fontsize=10,vector_friendly=True,figsize=(3,3))
sc.settings.verbosity = 3

Setting environment seed: 42


/home/comp/cszrwang/anaconda3/envs/SPACEL/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Global seed set to 0


In [3]:
sc_ad = sc.read_h5ad('/home/comp/cszrwang/data/Breast/cytospace/breast_sc_2.h5ad')
st_ad = sc.read_h5ad('/home/comp/cszrwang/data/Breast/cytospace/breast_st_2.h5ad')

sc.pp.filter_genes(st_ad,min_cells=1)
sc.pp.filter_genes(sc_ad,min_cells=1)
sc.pp.filter_cells(st_ad,min_genes=1)
sc.pp.filter_cells(sc_ad,min_genes=1)

In [4]:
sc_ad.obs['celltype']

CID3921_AAAGATGGTGAGGCTA     Endothelial cells
CID3921_AAAGATGTCCATGAGT     Endothelial cells
CID3921_AACCATGCAGTAACGG     Endothelial cells
CID3921_AACCGCGGTCAGAATA     Endothelial cells
CID3921_AACGTTGTCAGCTGGC     Endothelial cells
                                   ...        
CID45171_CTGCTGTTCCTTTCGG     Epithelial cells
CID3921_TCTTTCCTCCGAACGC      Epithelial cells
CID45171_TATTACCTCGACCAGC     Epithelial cells
CID3921_CTCATTATCTGTCTAT      Epithelial cells
CID3921_GATCGTACACAGATTC      Epithelial cells
Name: celltype, Length: 3977, dtype: category
Categories (11, object): ['B cells', 'Dendritic cells', 'Endothelial cells', 'Epithelial cells', ..., 'PCs', 'PVL', 'T cells CD4', 'T cells CD8']

In [5]:
spoint_model = Spoint.init_model(sc_ad,st_ad,celltype_key='celltype',deg_method='t-test',sm_size=100000,use_gpu=True)

Setting global seed: 42
normalizing counts per cell
    finished (0:00:00)
normalizing counts per cell
    finished (0:00:00)
### Finding marker genes...
ranking genes
    finished: added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:00:05)
celltype
B cells                      200
Dendritic cells              200
Endothelial cells            200
Epithelial cells             200
Fibroblasts                  200
Monocytes and Macrophages    200
NK cells                     200
PCs                          200
PVL                          200
T cells CD4                  200
T cells CD8                  200
Name: count, dtype: int64
### Used gene numbers: 1850
### Initializing sample probability


In [6]:
spoint_model.train(max_steps=5000, batch_size=1024)

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Epoch 100/100: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 100/100 [02:02<00:00,  1.23s/it, loss=1.28e+03, v_num=1]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 100/100: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 100/100 [02:02<00:00,  1.23s/it, loss=1.28e+03, v_num=1]


Step 5000: Test inference loss=-0.907: 100%|████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [28:14<00:00,  2.95it/s]


In [7]:
pre = spoint_model.deconv_spatial()

In [8]:
pre

,Epithelial cells,Monocytes and Macrophages,T cells CD4,T cells CD8,NK cells,Fibroblasts,Endothelial cells,B cells,PVL,PCs,Dendritic cells
AAACAAGTATCTCCCA.1__50x102,0.729145,0.231346,0.014374,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
AAACACCAATAACTGC.1__59x19,0.094729,0.158855,0.076552,0.068474,0.015142,0.094100,0.116020,0.060175,0.000000,0.124194,0.185591
AAACAGAGCGACTCCT.1__14x94,0.255785,0.025276,0.026709,0.038504,0.041021,0.237220,0.000000,0.000000,0.000000,0.325953,0.033706
AAACAGCTTTCAGAAG.1__43x9,0.000000,0.289943,0.000000,0.000000,0.010847,0.262344,0.244471,0.096421,0.000000,0.044249,0.022704
AAACAGGGTCTATATT.1__47x13,0.000000,0.028347,0.113066,0.115978,0.145224,0.155487,0.000000,0.197707,0.022245,0.207311,0.012437
...,...,...,...,...,...,...,...,...,...,...,...
TTGTTCAGTGTGCTAC.1__24x64,0.162033,0.151350,0.057331,0.140500,0.141493,0.156766,0.000000,0.012659,0.000000,0.154338,0.000000
TTGTTGTGTGTCAAGA.1__31x77,0.036812,0.018444,0.193569,0.097299,0.115253,0.083989,0.102238,0.140829,0.021322,0.105482,0.084763
TTGTTTCACATCCAGG.1__58x42,0.109209,0.152154,0.091729,0.113005,0.050500,0.124938,0.129072,0.020840,0.059289,0.141890,0.000000
TTGTTTCATTAGTCTA.1__60x30,0.000000,0.170371,0.131898,0.106210,0.043260,0.148943,0.195386,0.031944,0.047430,0.100146,0.021813


In [9]:
output_file_path = "/home/comp/cszrwang/results/Deconvolution/results/Breast_2/SPACEL/"

In [10]:
pre.to_csv(output_file_path + '/SPACEL_result.csv')

In [12]:
st_ad_show = spoint_model.st_ad

In [13]:
st_ad_show

AnnData object with n_obs × n_vars = 737 × 27
    obs: 'index', 'x', 'y', 'n_genes', 'batch', 'L4/5 IT CTX', 'L2/3 IT CTX-1', 'L6 IT CTX', 'L6b CTX', 'L6 CT CTX', 'Vip', 'L5 IT CTX', 'L5 NP CTX', 'Lamp5', 'Sst', 'L5 PT CTX'
    var: 'n_cells'
    uns: 'celltypes'
    layers: 'norm'